In [3]:
import cv2
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os


torch.cuda.is_available()

## See GPU
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")
    
# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)
model.eval()

# Define preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to extract features
def extract_features(image):
    image = preprocess(image)
    image = image.unsqueeze(0)  # add a batch dimension
    with torch.no_grad():
        features = model(image)
    return features

# Function to compare features for similarity
def compare_faces(feature1, feature2):
    similarity = torch.nn.functional.cosine_similarity(feature1, feature2)
    return similarity.item()

# Function to load features for a person from a directory
def load_person_features(directory):
    features = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            path = os.path.join(directory, filename)
            image = Image.open(path)
            features.append(extract_features(image))
    return torch.mean(torch.cat(features, dim=0), dim=0, keepdim=True)

# Load features for each known person
people_features = {
    'Yousef': load_person_features(r'C:\Users\Blank\Desktop\Git\MLProject\FR\data\Yousef'),
    'Ted': load_person_features(r'C:\Users\Blank\Desktop\Git\MLProject\FR\data\Ted'),
    'Mai': load_person_features(r'C:\Users\Blank\Desktop\Git\MLProject\FR\data\Mai'),
    'Isma': load_person_features(r'C:\Users\Blank\Desktop\Git\MLProject\FR\data\Isma'),
}

# Initialize webcam
cap = cv2.VideoCapture(0)

# Adjustable similarity threshold
similarity_threshold = 0.8

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the captured frame to PIL Image
    frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame_features = extract_features(frame_image)

    # Compare with each known person's features
    best_match = None
    highest_similarity = 0
    for name, features in people_features.items():
        similarity = compare_faces(features, frame_features)
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = name

    if highest_similarity > similarity_threshold:
        cv2.putText(frame, f"Match: {best_match}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        cv2.putText(frame, "No match found", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        if best_match == 'Isma':
            # Save the image for future retraining
            cv2.imwrite(f'isma_missed_{np.random.randint(10000)}.jpg', frame)

    cv2.imshow('Real-time Verification', frame)

    # Key to increase similarity threshold
    if cv2.waitKey(1) & 0xFF == ord('i'):
        similarity_threshold += 0.01
        print(f"Similarity threshold increased to {similarity_threshold}")

    # Key to decrease similarity threshold
    if cv2.waitKey(1) & 0xFF == ord('d'):
        similarity_threshold -= 0.01
        print(f"Similarity threshold decreased to {similarity_threshold}")

    # Exit loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

CUDA is available. Using GPU.
